In [1]:
#import relevant packages and libraies
import os
import re
import cv2
import numpy as np
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import tensorflow.keras as keras
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import models, layers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay


In [2]:
situations = {
                'c0': 'Safe driving', 
                'c1': 'Texting - right', 
                'c2': 'Talking on the phone - right', 
                'c3': 'Texting - left', 
                'c4': 'Talking on the phone - left', 
                'c5': 'Operating the radio', 
                'c6': 'Drinking', 
                'c7': 'Reaching behind', 
                'c8': 'Hair and makeup', 
                'c9': 'Talking to passenger'
}

In [ ]:
X = []
y = []
classes = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
base_path = './data/train/'

for i, target in enumerate(classes):
    
    files = os.listdir(base_path+target)
    
    print(f'we are now in {target} class')
    
    for file in files:
        
        # load the image
        img = load_img(base_path+target + '/' + file, target_size=(224, 224))
        
        #convert it to an array
        img_array = np.array(img)
              
        # append the array to X
        X.append(img_array)
        
        # append the numeric target to y
        y.append(i) 
        
print('finished')

we are now in c0 class
we are now in c1 class
we are now in c2 class
we are now in c3 class
we are now in c4 class
we are now in c5 class
we are now in c6 class
we are now in c7 class


In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
# shuffle the data
shuffler = np.random.permutation(len(X))
X = X[shuffler]
y = y[shuffler]

In [ ]:
# train-test split
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
#keep a copy of y test as the actual number labels for further process
y_test_true = y_test.copy()

In [ ]:
#one hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
#scaling
X_train = X_train.astype('float32')/255

In [ ]:
X_test = X_test.astype('float32')/255

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

# CNN

In [ ]:
#clear session 
K.clear_session()

In [ ]:
# number of possible label values
nb_classes = 10

#designing the layers and build model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(224,224,3),padding = 'same'))
model.add(BatchNormalization())

#model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(224,224,3),padding = 'same'))
#model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same'))
model.add(BatchNormalization())

#model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same'))
#model.add(BatchNormalization())

model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same'))
model.add(BatchNormalization())


#model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same'))
#model.add(BatchNormalization())


model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128,kernel_size = (3,3),activation = 'relu',padding = 'same'))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Dense(nb_classes, activation='softmax'))

In [ ]:
#check the summary:
model.summary()

In [ ]:
#compile the model
model.compile(optimizer=SGD(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#define early stopping for fitting
early_stop = keras.callbacks.EarlyStopping(
                                            monitor='val_loss', 
                                            min_delta=0.0005, 
                                            patience=10,
                                            verbose=1, 
                                            mode='auto',
                                           )

In [ ]:
datagen = ImageDataGenerator(
                            height_shift_range=0.5,
                            width_shift_range = 0.5,
                            zoom_range = 0.5,
                            rotation_range=30
                            )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = 32)

# Fits the model on batches with real-time data augmentation:
history = model.fit_generator(
                               data_generator,
                               steps_per_epoch=len(X_train)/32, 
                               callbacks=[early_stop],
                               epochs = 50,
                               validation_data = (X_test, y_test)
                              )


In [ ]:
model.save('./models/model_cnn_sgd_scaling_full_2.h5')

In [ ]:
ypred = model.predict(X_test)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 12,20
# have a look at the data
for i in range(10):
    plt.subplot(5, 2, i+1)
    plt.imshow(X_test[i])
    plt.axis('off')
    plt.title(situations[classes[np.argmax(ypred[i])]]) 

# Examine the model

In [ ]:
#plot accuracy curve
plt.figure(figsize=(8,6))
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label= 'validation accuracy')

plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()


In [ ]:
# Plot loss
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [ ]:
true_list =[]
for i in range(len(y_test_true)):
    if np.argmax(ypred[i])==y_test_true[i]:
        true_list.append(i)
        
probability = len(true_list) / len(y_test_true)
round(probability,3)

In [ ]:
#plot confusion matrix
cm = confusion_matrix(y_true=y_test_true, y_pred=np.argmax(ypred, axis=1))

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=classes)
fig, ax = plt.subplots(figsize=(10,10))

disp.plot(ax=ax);

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 12,20
# have a look at the data
for i in range(10):
    plt.subplot(5, 2, i+1)
    plt.imshow(X_test[i])
    plt.axis('off')
    plt.title(situations[classes[np.argmax(ypred[i])]]) 

In [ ]:
history.history

In [ ]:
history_x = {'loss': [0.6788745522499084,
  0.09309027343988419,
  0.04502665624022484,
  0.028193367645144463,
  0.01988980919122696,
  0.015026142820715904,
  0.01297104824334383,
  0.010552069172263145,
  0.009022541344165802,
  0.008177461102604866,
  0.007196308579295874,
  0.006253139581531286,
  0.005592679604887962,
  0.005200664512813091,
  0.0048648216761648655,
  0.004325977526605129,
  0.004437521565705538,
  0.0038987549487501383,
  0.003602318000048399,
  0.0035539239179342985,
  0.0031619674991816282,
  0.0031310778576880693,
  0.0030073411762714386,
  0.0027063859160989523,
  0.0028454300481826067,
  0.0026335103902965784,
  0.0024933728855103254,
  0.0023065018467605114,
  0.002182203810662031,
  0.0021467802580446005,
  0.0021780335810035467,
  0.0019917781464755535,
  0.0020393950399011374,
  0.0018913195235654712,
  0.0018716666381806135,
  0.001779830316081643,
  0.0017926986329257488,
  0.0017754330765455961,
  0.0016076870961114764,
  0.001544486847706139,
  0.0016306579345837235,
  0.0014818470226600766,
  0.0014596282271668315,
  0.0014299782924354076,
  0.0013975645415484905,
  0.001439416198991239,
  0.0013373905094340444,
  0.001357017899863422,
  0.0012831803178414702,
  0.0012083920883014798],
 'accuracy': [0.802437961101532,
  0.9842426180839539,
  0.994945764541626,
  0.9979188442230225,
  0.9986621141433716,
  0.9994797110557556,
  0.9994053840637207,
  0.9996283650398254,
  0.9996283650398254,
  1.0,
  0.9997770190238953,
  0.9998513460159302,
  1.0,
  0.9999256730079651,
  0.9999256730079651,
  1.0,
  0.9998513460159302,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'val_loss': [2.4285008907318115,
  0.5887197256088257,
  0.1103152334690094,
  0.073394276201725,
  0.06286414712667465,
  0.05797842517495155,
  0.057778436690568924,
  0.04893775284290314,
  0.08751919120550156,
  0.04422752931714058,
  0.042050883173942566,
  0.040523283183574677,
  0.041227638721466064,
  0.039321884512901306,
  0.03771194815635681,
  0.035737045109272,
  0.036077626049518585,
  0.036514002829790115,
  0.0332961343228817,
  0.033254560083150864,
  0.032514166086912155,
  0.03254510462284088,
  0.03128845989704132,
  0.030809486284852028,
  0.03277873992919922,
  0.030335091054439545,
  0.030178410932421684,
  0.030333690345287323,
  0.030870309099555016,
  0.02990068681538105,
  0.02908347360789776,
  0.02898188680410385,
  0.02818087860941887,
  0.027558594942092896,
  0.027215182781219482,
  0.027875814586877823,
  0.0267275869846344,
  0.027197787538170815,
  0.027168041095137596,
  0.026311203837394714,
  0.02606181986629963,
  0.02639640122652054,
  0.025416001677513123,
  0.026084549725055695,
  0.025617264211177826,
  0.02574216015636921,
  0.025612205266952515,
  0.025226403027772903,
  0.0251119676977396,
  0.02525498904287815],
 'val_accuracy': [0.2729097008705139,
  0.8276476860046387,
  0.9839465022087097,
  0.9875139594078064,
  0.9899665713310242,
  0.9897435903549194,
  0.9899665713310242,
  0.9919732213020325,
  0.9797101616859436,
  0.9917502999305725,
  0.9933110475540161,
  0.9928650856018066,
  0.9930880665779114,
  0.9933110475540161,
  0.9937569499015808,
  0.9937569499015808,
  0.9933110475540161,
  0.9933110475540161,
  0.9935340285301208,
  0.9939799308776855,
  0.9939799308776855,
  0.9937569499015808,
  0.9939799308776855,
  0.9939799308776855,
  0.9935340285301208,
  0.9935340285301208,
  0.9937569499015808,
  0.9942029118537903,
  0.9939799308776855,
  0.9937569499015808,
  0.9939799308776855,
  0.9942029118537903,
  0.994425892829895,
  0.994425892829895,
  0.9948717951774597,
  0.994425892829895,
  0.9948717951774597,
  0.9948717951774597,
  0.994648814201355,
  0.994648814201355,
  0.994425892829895,
  0.9948717951774597,
  0.9948717951774597,
  0.9950947761535645,
  0.9950947761535645,
  0.9950947761535645,
  0.9950947761535645,
  0.9953176975250244,
  0.9950947761535645,
  0.9950947761535645]}